# Exercise #

In this exercise, you'll build a model to predict hotel cancellations with a binary classifier.

In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

First, load the *Hotel Cancellations* dataset. (**fix the path** if necessary)

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

hotel = pd.read_csv('data/hotel_train.csv')

In [ ]:
''' per vedere quante sono le prenotazioni cancellate e quante no, per farsi un idea di quanto
sono sbilanciate'''
hotel.is_canceled.value_counts() 
# per vedere la distribuzione dei valori nelle varie colonne
numerical = hotel.loc[:, hotel.dtypes == int or hotel.dtypes == float].columns.to_list()
display(numerical)

In [ ]:
''' per ogni colonna descrivo dati come il numero di istanze, media, media, varianza etc
per identificare anche eventuali anomalie'''
for col in numerical:
    display(hotel[col].describe())
    print("")

Eventualmente posso fare *feature engineering* per cercare di combinare feature 
per crearne un'altra piú significativa ad esempio in questo caso combinare
date e lead_time per creare una nuova feature che indica il numero di giorni di permanenza eliminando
giorni feriali e giorni festivi, oppure aggregare la colonna children e babies in una colonna sola
o magari cambiare il conteggio delle prenotazioni cancellate in un indicatore binario

In [ ]:
hotel.tight_layout() # per graficare i dati

In [ ]:
''' Oppure verificare se esiste correlazione lineare tra le feature'''
import seaborn as sns
heatmap = sns.heatmap(hotel.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12)
display(heatmap)

In [ ]:
X = hotel.copy()
y = X.pop('is_canceled') # this is the target column

# enumerate months
X['arrival_date_month'] = \
    X['arrival_date_month'].map(
        {'January':1, 'February': 2, 'March':3,
         'April':4, 'May':5, 'June':6, 'July':7,
         'August':8, 'September':9, 'October':10,
         'November':11, 'December':12}
    )

features_num = [
    "lead_time", "arrival_date_week_number",
    "arrival_date_day_of_month", "stays_in_weekend_nights",
    "stays_in_week_nights", "adults", "children", "babies",
    "is_repeated_guest", "previous_cancellations",
    "previous_bookings_not_canceled", "required_car_parking_spaces",
    "total_of_special_requests", "adr",
]
features_cat = [
    "hotel", "arrival_date_month", "meal",
    "market_segment", "distribution_channel",
    "reserved_room_type", "deposit_type", "customer_type",
]

# preparo la pipeline per trasformare feature numeriche
transformer_num = make_pipeline(
    SimpleImputer(strategy="constant"), # there are a few missing values, default fill with 0
    StandardScaler(), #Standardizzare le feature
)

# preparo la pipeline per trasformare feature categoriche
transformer_cat = make_pipeline(
    SimpleImputer(strategy="constant", fill_value="NA"), # fill missing values with NA
    OneHotEncoder(handle_unknown='ignore'), # codificare i valori categorici con enum
)

# Metto insieme le pipeline 
preprocessor = make_column_transformer(
    (transformer_num, features_num),
    (transformer_cat, features_cat),
)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.75)
''' dato che le classi sono un po sbilanciate potrei usare stratify per bilanciarle che potrei anche
non fare se uso cross validation'''
#X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.75)

# butto i dati nella pipeline e li trasformo
X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)

input_shape = [X_train.shape[1]]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE: define the model given in the diagram
model = ...

